In [1]:
import os
import re
import random
import json
from pathlib import Path
import copy
import warnings
from collections import defaultdict
import datetime
from tqdm import tqdm

import numpy as np
import pandas as pd

import yake
import spacy
from spellchecker import SpellChecker

import torch
import torch.nn as nn

from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer
from transformers import AutoConfig
from transformers import PreTrainedModel, PreTrainedTokenizer
from transformers import GenerationConfig

from safetensors.torch import load_file


## Data import

In [178]:
# Data import

data_folder = Path.cwd().joinpath('Data')

tickets_df = pd.read_excel(data_folder.joinpath('tickets_cleaned.xlsx'))
tickets_history_df = pd.read_excel(data_folder.joinpath('ticket_history.xlsx'))

with open(data_folder.joinpath('ticket_template_query.txt')) as tq, \
    open(data_folder.joinpath('ticket_template_answer.txt')) as ta, \
        open(data_folder.joinpath('ticket_template_his.txt')) as th:

        query_template = tq.read()
        answer_template = ta.read()
        history_template = th.read()

tickets_df.head()


,TICKETID,ALTERNATEKEYSUFFIX,AREA,CATEGORY,ISSUE,RECEIVEDDATE,COMPLETEDDATE,SUBJECT,URGENCYCODE,PROBLEM,SOLUTION,RESULTIONSUMMARY,ACCOUNTID,ACCOUNT
0,t6UJ9A00GHH8,NaN,NaN,S1000v4,NaN,2025-05-23 11:30:32,2025-05-23 16:05:51,URGENT HELP NEEDED: PI 7268 Batch posting fail...,3,Would you be able to help correct the whatever...,Batch had been imported incorrectly. Corrected...,NaN,A6UJ9A0023H4,Account_1
1,t6UJ9A00GFTK,NaN,NaN,S1000v4,NaN,2025-05-08 14:38:09,2025-05-23 15:11:47,MUK - WIP Report,3,"Hi, \n\nPlease see attachment of our report t...",I advised to run the report in WOP Reports w...,NaN,A6UJ9A0009N9,Account_2
2,t6UJ9A00GGSG,NaN,NaN,S1000v4,NaN,2025-05-19 07:02:04,2025-05-23 14:39:32,invoices into the GL,3,"Hi,\n\nThere is a process that my team do and ...",user is not set up as a pop user so currently ...,NaN,A6UJ9A000H3R,Account_3
3,t6UJ9A00GHDL,NaN,NaN,S1000v4,BACS,2025-05-22 14:37:44,2025-05-23 14:34:00,BACS query,1,"when I raise a BACS, does it have to be raised...",Advised that the remittance date entered when ...,NaN,A6UJ9A0083J7,Account_4
4,t6UJ9A00GHHG,NaN,NaN,S1000v4,NaN,2025-05-23 12:20:29,2025-05-23 12:56:26,NaN,3,",\n\nCan you please set up as a new timesheet...",set up via Contracting - -,NaN,A00SWA300000,Account_5


## Data preprocessing

In [61]:
# Define data preprocessing functions

# Apply basic NLP preprocessing

def preprocess_ticket(text, spell_checker = None, keyword_extractor = None, spacy_nlp = None,
                         whitespace_cleaning = False, typo_correction = False,
                             keyword_extraction = False, ner = False):

    text_dict = {}
    
    # Whitespace cleanup

    if whitespace_cleaning:

        text = re.sub(r'\s+', lambda m: '\n' if '\n' in m.group() else ' ', text.strip())
    
    # Typo correction

    if typo_correction and spell_checker is not None:
        
        corrected_words = []
        for word in cleaned.split():
            # Keep numbers, codes, and certain technical terms unchanged
            if re.match(r'^[A-Za-z0-9._-]+$', word):
                corrected_words.append(spell_checker.correction(word) or word)
            else:
                corrected_words.append(word)
        corrected_text = ' '.join(corrected_words)
        text = corrected_text

    text_dict['text'] = text 
    
    # Keyword extraction

    if keyword_extraction and keyword_extractor is not None:
        
        keywords = [kw for kw, score in keyword_extractor.extract_keywords(text)]
        text_dict['keywords'] = ', '.join(keywords)
    
    # Named Entity Recognition

    if ner and spacy_nlp is not None:
        
        doc = spacy_nlp(text)
        entities = [ent.text + ' - ' + ent.label_ for ent in doc.ents]
        text_dict['entities'] = ', '.join(entities)
    
    return pd.Series(text_dict)

# Apply textual templates for structured query-answer pairs

def aggregate_history(df, hist_col, date_col, ticket_col, activity_type_col, template,
                      remove_tokens = True, ascending_hist = False, exclusion_list = ['Ticket Acknow']):

    # Ensure chronological sorting within each ticket and perform filtering
    
    df = df[~((df[activity_type_col] == 'E-mail') & (df[hist_col].str.len() <= 139))]
    df = df[~df[hist_col].isna()]
    df = df[~df[activity_type_col].isin(exclusion_list)].reset_index(drop = True)
    df = df.sort_values(by = [ticket_col, date_col], ascending = [True, ascending_hist])

    df[hist_col] = df[hist_col].fillna('').str.replace(r'(\S)\1{2,}', '', regex = True)
    df[hist_col] = df[hist_col].apply(lambda text: re.sub(r'\s+', lambda m: '\n' if '\n' in m.group() else ' ', str(text).strip()))
    
    # Apply the template to each row
    
    df['FORMATTEDDESC'] = df.apply(lambda x: template.format(
        created_date = x[date_col],
        activity = x[activity_type_col],
        a_desc = x[hist_col]), axis = 1)
    
    # Group by ticket and aggregate formatted texts with newline separation
    
    df_grouped_his = (
        df.groupby(ticket_col, as_index = False)['FORMATTEDDESC']
        .apply(lambda texts: '\n\n'.join(texts))
        .reset_index(drop = True)
    )

    if remove_tokens:

        pattern1 = r'(?:\[REDACTED_[A-Z]+\][\s\W]+){1,}\[REDACTED_[A-Z]+\]'
        pattern2 = r'E-mail to:.*?\S*#\s*\d+.*?has changed its status to\s+\w+'
        df_grouped_his['FORMATTEDDESC'] = df_grouped_his['FORMATTEDDESC'].fillna('').str.replace(pattern1, '', regex = True)
        df_grouped_his['FORMATTEDDESC'] = df_grouped_his['FORMATTEDDESC'].str.replace(r'(\n)\1{2,}', r'\1\1', regex = True)
        df_grouped_his['FORMATTEDDESC'] = df_grouped_his['FORMATTEDDESC'].str.replace(pattern2, '', regex = True)
        
    return df_grouped_his

# Define a function that prepares a dataset

def structure_tickets(tickets_df, his_tickets_df,
                      ticket_col, subject_col, cat_col, issue_type_col, received_date_col, query_col, query_template,
                      urgency_col, res_sum_col, solution_col, answer_template,
                      his_col, his_date_col, his_activity_type_col, his_template,
                      spell_checker = None, keyword_extractor = None, spacy_nlp = None,
                      ascending_hist = False, exclusion_list = ['Ticket Acknow']):

    tickets_df = tickets_df[~tickets_df[query_col].isna()].reset_index(drop = True)

    tickets_df[[query_col, query_col + '_keywords']] = \
        tickets_df[query_col].fillna('').apply(lambda x: preprocess_ticket(x,
            spell_checker = spell_checker, keyword_extractor = keyword_extractor, spacy_nlp = spacy_nlp,
            whitespace_cleaning = True, typo_correction = False, keyword_extraction = True, ner = False))

    tickets_df[[solution_col, solution_col + '_keywords']] = \
        tickets_df[solution_col].fillna('').apply(lambda x: preprocess_ticket(x,
            spell_checker = spell_checker, keyword_extractor = keyword_extractor, spacy_nlp = spacy_nlp,
            whitespace_cleaning = True, typo_correction = False, keyword_extraction = True, ner = False))

    his_tickets_agg_df = aggregate_history(his_tickets_df, his_col, his_date_col,
                                           ticket_col, his_activity_type_col, his_template,
                                           remove_tokens = True,
                                           ascending_hist = ascending_hist,
                                           exclusion_list = exclusion_list)

    tickets_df = tickets_df.merge(his_tickets_agg_df, on = ticket_col, how = 'left')

    tickets_df['STRUCTURED' + query_col] = \
        tickets_df.apply(lambda x: query_template.format(
            tid = x[ticket_col],
            subject = x[subject_col],
            cat = x[cat_col],
            issue_type = x[issue_type_col],
            received_date = x[received_date_col],
            keywords = x[query_col + '_keywords'],
            problem = x[query_col]), axis = 1)

    tickets_df['STRUCTURED' + solution_col] = \
        tickets_df.apply(lambda x: answer_template.format(
            urgency = x[urgency_col],
            res_sum = x[res_sum_col],
            keywords = x[solution_col + '_keywords'],
            solution = x[solution_col],
            ticket_status_his = x['FORMATTEDDESC']), axis = 1)

    return tickets_df[[ticket_col, query_col, solution_col, 'STRUCTURED' + query_col, 'STRUCTURED' + solution_col]]


In [62]:
# Apply transformations and save the data

keyword_extractor = yake.KeywordExtractor(n = 2, top = 10)

processed_tickets_df = \
    structure_tickets(tickets_df, tickets_history_df,
        ticket_col = 'TICKETID', subject_col = 'SUBJECT', cat_col = 'CATEGORY',
        issue_type_col = 'ISSUE', received_date_col = 'RECEIVEDDATE', query_col = 'PROBLEM',
        urgency_col = 'URGENCYCODE', res_sum_col = 'RESULTIONSUMMARY', solution_col = 'SOLUTION',
        his_col = 'ACTIVITYDESC', his_date_col = 'CREATEDATE', his_activity_type_col = 'ACTIVITYTYPE',
        query_template = query_template, answer_template = answer_template, his_template = history_template,
        keyword_extractor = keyword_extractor, ascending_hist = False, exclusion_list = ['Ticket Acknow'])

processed_tickets_df.to_json(data_folder.joinpath('processed_tickets_df.json'),
                                 orient = 'index', double_precision = 15, index = True)


In [63]:
processed_tickets_df.head()

,TICKETID,PROBLEM,SOLUTION,STRUCTUREDPROBLEM,STRUCTUREDSOLUTION
0,t6UJ9A00GHH8,Would you be able to help correct the whatever...,Batch had been imported incorrectly. Corrected...,Ticket metadata:\n\n Ticket ID: t6UJ9A00GHH8\...,Solution metadata:\n \n Urgency code: 3\n Re...
1,t6UJ9A00GFTK,"Hi,\nPlease see attachment of our report that ...",I advised to run the report in WOP Reports whi...,Ticket metadata:\n\n Ticket ID: t6UJ9A00GFTK\...,Solution metadata:\n \n Urgency code: 3\n Re...
2,t6UJ9A00GGSG,"Hi,\nThere is a process that my team do and I ...",user is not set up as a pop user so currently ...,Ticket metadata:\n\n Ticket ID: t6UJ9A00GGSG\...,Solution metadata:\n \n Urgency code: 3\n Re...
3,t6UJ9A00GHDL,"when I raise a BACS, does it have to be raised...",Advised that the remittance date entered when ...,Ticket metadata:\n\n Ticket ID: t6UJ9A00GHDL\...,Solution metadata:\n \n Urgency code: 1\n Re...
4,t6UJ9A00GHHG,",\nCan you please set up as a new timesheet re...",set up via Contracting - -,Ticket metadata:\n\n Ticket ID: t6UJ9A00GHHG\...,Solution metadata:\n \n Urgency code: 3\n Re...


## LLM-based processing

In [48]:
# Define loading function

def load_model(model_path, device, load_8bit = False, from_pretrained = True, is_dummy = False,
                   dummy_parameters = {}, rope_scaling = None, max_context_length = None, rope_theta = None):

    tokenizer = AutoTokenizer.from_pretrained(model_path, local_files_only = True)

    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id

    gen_config = GenerationConfig.from_pretrained(model_path)

    if type(max_context_length) == int:
        setattr(gen_config, 'max_length', max_context_length)

    if from_pretrained:
    
        model = AutoModelForCausalLM.from_pretrained(
            model_path,
            device_map = 'auto' if device == torch.device('cuda') else None,
            torch_dtype = torch.bfloat16 if device == torch.device('cuda') else torch.float32,
            load_in_8bit = load_8bit,
            local_files_only = True,
            use_safetensors = True,
            rope_scaling = rope_scaling
        ).to(device)

    else:

        # Pulling out the config

        config = AutoConfig.from_pretrained(model_path, local_files_only = True, use_safetensors = True)
        strict_loading = not is_dummy
        
        # Optional config editing for a dummy model
        
        if is_dummy:
            for name, value in dummy_parameters.items():
                setattr(config, name, value)

        if type(rope_scaling) == dict:
            setattr(config, 'rope_scaling', rope_scaling)

        if type(max_context_length) == int:
            setattr(config, 'max_position_embeddings', max_context_length)

        if rope_theta is not None:
            setattr(config, 'rope_theta', rope_theta)
        
        # Instantiate the model
        
        model = AutoModelForCausalLM.from_config(config).to(device)
        
        # Load parameters
        
        with open(model_path.joinpath('model.safetensors.index.json'), 'r') as file:
            index = json.load(file)
        
        safetensors_params_map = defaultdict(list)
        for param_name, safetensor_name in index['weight_map'].items():
            safetensors_params_map[safetensor_name].appendf(param_name)

        full_state_dict = {}
        
        for safetensor_name, param_names in safetensors_params_map.items():
            safetensor_path = model_path.joinpath(safetensor_name)
            shard_dict = load_file(safetensor_path)
            model_dict = model.state_dict()
        
            shard_dict = {
                key : value for key, value in shard_dict.items()
                if key in model_dict and model_dict[key].shape == value.shape
            }

            full_state_dict.update(shard_dict)
        
        with warnings.catch_warnings(action = 'ignore'):
            model.load_state_dict(full_state_dict, strict = strict_loading)

        # Parallelize if possible

        if torch.cuda.device_count() > 1:
            model = model.to(device)
            model = nn.DataParallel(model)

    return tokenizer, model, gen_config
    

# Define batch inference function

def batch_inference_llama(model, tokenizer, df, query_column, answer_column, instruction_prompt, gen_config,
                              examples = None, batch_size = 4, max_new_tokens = 1024, num_examples = 1):
    
    results = []
    max_context_len = gen_config.max_length

    orig_model = model.module if hasattr(model, 'module') else model
    
    # Iterate over batches
    
    for start in tqdm(range(0, len(df), batch_size)):

        conversation = []
        batch_texts_truncated = []
        
        # Build few-shot prompt

        random.shuffle(examples)
        
        if examples:
            for i in range(num_examples):
                conversation.append({'role' : 'user', 'content' : examples[i][0]})
                conversation.append({'role' : 'assistant', 'content' : examples[i][1]})
    
        # System role
        
        conversation.insert(0, {'role' : 'system', 'content' : instruction_prompt})
    
        # Tokenize static context
        
        static_context_tokens = tokenizer.apply_chat_template(
            conversation = conversation,
            add_generation_prompt = True,
            tokenize = True
        )
        static_token_count = len(static_context_tokens)

        
        batch_texts = (df[query_column].iloc[start:start+batch_size] + '\n\n' +
                           df[answer_column].iloc[start:start+batch_size]).tolist()

        # Ensure a correct context length

        for text in batch_texts:

            text_tokens = tokenizer(text.strip(), return_tensors = 'pt')
            text_len = text_tokens.input_ids.shape[1]

            # Check if adding this text would exceed max context
    
            if static_token_count + text_len + max_new_tokens > max_context_len:

                allowed_len = max_context_len - static_token_count - max_new_tokens
                truncated_ids = text_tokens.input_ids[:, :allowed_len]
                truncated_text = tokenizer.decode(truncated_ids[0], skip_special_tokens = True)
                batch_texts_truncated.append(truncated_text)
                
            else:
                
                batch_texts_truncated.append(text)
        
        # Build prompts for batch
        
        prompts = [
            tokenizer.apply_chat_template(
                conversation = conversation + [{'role' : 'user', 'content': text.strip()}],
                add_generation_prompt = True,
                tokenize = False) \
            for text in batch_texts_truncated
        ]
        
        # Tokenize
        
        inputs = tokenizer(prompts, return_tensors = 'pt',
                               padding = True, padding_side = 'left',
                                   truncation = False).to(orig_model.device)
        
        # Generate outputs

        orig_model.eval()
        
        with torch.no_grad():
            outputs = orig_model.generate(
                **inputs,
                pad_token_id = tokenizer.pad_token_id,
                generation_config = gen_config,
                return_dict_in_generate = True,
                use_cache = True,
                max_new_tokens = max_new_tokens
            )
        
        decoded_outputs = tokenizer.batch_decode(outputs.sequences, skip_special_tokens = False)
        
        # Extract only the part after assistant generation
        
        cleaned_outputs = []
        for full_text in decoded_outputs:
            if '<|start_header_id|>assistant<|end_header_id|>' in full_text:
                cleaned_outputs.append(full_text.split('<|start_header_id|>assistant<|end_header_id|>')[-1].strip())
            else:
                cleaned_outputs.append(full_text.strip())
        
        results.extend(cleaned_outputs)
    
    df = df.copy()
    df['SUMMARIZEDSOLUTION'] = results
    
    return df

# Optional function for a tokenized input length calculation

def calculate_token_length(tokenizer, df, query_column, answer_column, instruction_prompt,
                               examples = None, batch_size = 4, long_text_threshold = 4000):
    
    results = []
    conversation = []
    
    # Build few-shot prompt
    
    if examples:
        for inp, out in examples:
            conversation.append({'role' : 'user', 'content' : inp})
            conversation.append({'role' : 'assistant', 'content' : out})

    # System role
    
    conversation.insert(0, {'role' : 'system', 'content' : instruction_prompt})
    
    # Iterate over batches
    
    for start in range(0, len(df), batch_size):
        
        batch_texts = (df[query_column].iloc[start:start+batch_size] + '\n\n' +
                           df[answer_column].iloc[start:start+batch_size]).tolist()
        
        # Build prompts for batch
        
        prompts = [
            tokenizer.apply_chat_template(
                conversation = conversation + [{'role' : 'user', 'content': re.sub(r'\s+', ' ',text).strip()}],
                add_generation_prompt = True,
                tokenize = False) \
            for text in batch_texts
        ]
            
        # Tokenize
        
        inputs = tokenizer(prompts, return_tensors = 'pt', padding = True, truncation = False, padding_side = 'left').to(DEVICE)
    
        length = inputs['attention_mask'].sum(axis = 1).tolist()
    
        results.extend(length)

    df['combined_case_token_length'] = np.array(results)
    df['is_long'] = df['combined_case_token_length'] >= long_text_threshold

    df = df.copy()

    return df
    

In [30]:
# Define model configuration and import parameters

FROM_PRETRAINED = False
IS_DUMMY = True
DUMMY_PARAMETERS = {'hidden_size' : 2, 'intermediate_size' : 4, 'head_dim' : 8}

USE_GPU = False
DEVICE = torch.device('mps' if torch.mps.is_available() and USE_GPU else ('cuda:0' if torch.cuda.is_available() and USE_GPU else 'cpu'))

MODEL_DICT = \
    {'name' : 'Llama-3-8B-Instruct',
     'repo_id' : 'meta-llama/Meta-Llama-3-8B-Instruct',
     'required_files' : ['config.json', 'generation_config.json', 'model.safetensors',
                         'model-00001-of-00004.safetensors', 'model-00002-of-00004.safetensors',
                         'model-00003-of-00004.safetensors', 'model-00004-of-00004.safetensors',
                         'special_tokens_map.json', 'tokenizer.json', 'tokenizer_config.json', 'model.safetensors.index.json'],
     'model_path' : ['LLaMa', '3.1-8B-Instruct']}

# Model and data paths 

llm_path = Path.cwd().joinpath(*MODEL_DICT['model_path'])
data_folder = Path.cwd().joinpath('Data')

# Upload data

processed_tickets_df = pd.read_json(data_folder.joinpath('processed_tickets_df.json'),
                                        orient = 'index', typ = 'frame',
                                            dtype = str, precise_float = True)

# Upload auxiliary files 

with open(data_folder.joinpath('instruction_prompt_preprocessing.txt')) as ip, \
    open(data_folder.joinpath('examples.txt')) as e:
        
    instruction_prompt = ip.read()
    examples = e.read()

examples = [(example.split('|<[sep1]>|')[0].strip(), example.split('|<[sep1]>|')[1].strip())
                for example in examples.split('|<[sep2]>|')]

# Upload a model

tokenizer, llm_model, gen_config = \
    load_model(llm_path, DEVICE,
               load_8bit = False, from_pretrained = False,
               is_dummy = True, dummy_parameters = DUMMY_PARAMETERS,
               rope_scaling = {'type': 'linear', 'factor': 4.0}, max_context_length = 32768)


In [31]:
# Apply data transformation

processed_tickets_df = \
    batch_inference_llama(llm_model, tokenizer, processed_tickets_df,
                          'STRUCTUREDPROBLEM', 'STRUCTUREDSOLUTION',
                          instruction_prompt, gen_config,
                          examples = examples, batch_size = 4, max_new_tokens = 1024, num_examples = 3)

processed_tickets_df.to_json(data_folder.joinpath('llm_processed_tickets_df.json'),
                                 orient = 'index', double_precision = 15, index = True)


100%|████████████████████████████████████████████| 3/3 [07:02<00:00, 140.89s/it]


In [64]:
# Optionally execute a check for tokenized input lengths

processed_tickets_length_df = \
    calculate_token_length(tokenizer, processed_tickets_df,
                           'STRUCTUREDPROBLEM', 'STRUCTUREDSOLUTION',
                           instruction_prompt, examples = examples, batch_size = 4,
                           long_text_threshold = 4000)

processed_tickets_length_df.to_excel(data_folder.joinpath('combined_case_token_lengths.xlsx'))
    